Applying Optimization and Grid Search

In [35]:
#imports

import itertools
import pandas as pd
#from library.algorithms.genetic_algorithms.algorithm import genetic_algorithm
from library_group.ours_genetic_algo_func import genetic_algorithm
from library_group.lineup import LUGASolution
from library_group.mutation_funcs import block_rotation_mutation, two_phase_shuffle_mutation, semi_shuffle
from library_group.xo_funcs import cyclic_crossover, custom_pmxo

#Line-up Problem: Datasets
from library_group.lineup import artists_df, conflicts_df

#from library_group.
from library.algorithms.genetic_algorithms.selection import fitness_proportionate_selection #NÃO PODEMOS USAR ERA SÓ PARA EXEMPLO

In [ ]:
"""
Esta célula não corre
"""

#o problema está no crossover???
random_solution = LUGASolution(mutation_function = block_rotation_mutation, crossover_function =  cyclic_crossover)
random_sol2 = LUGASolution(mutation_function = block_rotation_mutation, crossover_function =  cyclic_crossover)            
random_solution.crossover(random_sol2)

In [ ]:
"""
Esta célula não corre
"""

POP_SIZE = 50

initial_population = [LUGASolution(mutation_function = block_rotation_mutation, crossover_function = cyclic_crossover
                                   )
                      for i in range(POP_SIZE)]

genetic_algorithm(initial_population=initial_population,
                  max_gen=100,
                  selection_algorithm = fitness_proportionate_selection,
                  maximization=False,
                  xo_prob=0.8,
                  mut_prob=1,
                  elitism=True,
                  verbose=True)

In [ ]:
def comb_tuning(mut_func: dict, xo_func: dict, selection_algo: list,
                n_runs: int = 30, max_gen:int = 100, POP_SIZE: int = 50):
    
    """
    Returns a DataFrame with the following columns:
    - 'Combination' - Combination tuple of a mutation, a crossover and a selection algorithm
    - 'Run' - Each Combination is runned a specified amount of times, this column specifies the run 
    number in which the results were obtained.
    - 'Generation' - Each Run is composed of a specified amount of generations, this column specifies the 
    generation number in which the results were obtained.
    - 'Best Fitness' - This is the best fitness obtained at each generation.

    Purpose:
    With this df we can play around inter-run and intra-run statistics, since it contains 
    all the results from the various combinations
    """

    combinations = list(itertools.product(list(mut_func.keys()), list(xo_func.keys()), selection_algo))

    combs_results = []
    for comb in combinations:
        # Store the best results of each generation for each run.
        runs_results = []
        for run in range(1, n_runs+1):
            initial_population = [LUGASolution(mutation_function = comb[0], crossover_function = comb[1],
                                               values=values, weights=weights, capacity=capacity)
                                               for i in range(POP_SIZE)]
            
            _, generation_df = genetic_algorithm(initial_population=initial_population,
                    max_gen=max_gen,
                    selection_algorithm = comb[2],
                    maximization=False,
                    xo_prob=xo_func[comb[1]],
                    mut_prob= mut_func[comb[0]],
                    elitism=True,
                    verbose=False)
            
            current_run = pd.DataFrame()
            current_run['Run'] = [run for _ in range(1, n_runs+1)] #fill column 'Run' with the #run it is in
            
            #with run=#run, merge the df that contains the columns with #generation and its best fitness
            one_run = pd.concat([current_run, generation_df], axis=1) 
            runs_results.append(one_run)
        
        current_comb_all_runs = pd.concat(runs_results) #concat all the different runs along the index axis
        
        # Generate filename based on combination
        filename = f"comb_{comb[0]}_{comb[1]}_{comb[2]}.csv"
        # Save to CSV
        current_comb_all_runs.to_csv(filename, index=False)
        
        current_comb = pd.DataFrame()
        current_comb['Combination'] = [(comb[0], comb[1], comb[2]) for _ in range(len(combinations))]
        one_comb = pd.concat([current_comb, current_comb_all_runs], axis=1)
        combs_results.append(one_comb)
    
    final_results = pd.concat(combs_results)
    
    return final_results

In [ ]:
#Example
mut_func = {block_rotation_mutation:0.05, two_phase_shuffle_mutation: 0.1, semi_shuffle: 0.2} #, }
xo_func = {cyclic_crossover:0.7, custom_pmxo: 0.6} #, }
#selection_algo = [fitness_proportionate_selection, ] #USAR OS NOSSOS ALGORITMOS

In [ ]:
comb_tuning(mut_func=mut_func, xo_func=xo_func, selection_algo=selection_algo)